## Access the database

In [1]:
# define parameters
db_name = 'dbsprojekt' # db should already be created locally
user = 'postgres'
password = 'password5432'

import_done = True # nach dem ersten Ausführen auf True setzen

In [2]:
import pandas as pd
# import numpy as np
import psycopg2

In [3]:
# connect to database
try:
    print('Connecting to the PostgreSQL database...')
    conn = psycopg2.connect(host = 'localhost', database = db_name, user = user, password = password)
    
    cur = conn.cursor()
    
    print('connected')
    
except (Exception, psycopg2.DatabaseError) as error:
    print(error)

Connecting to the PostgreSQL database...
connected


## Read in data files and create tables

In [4]:
# read files
f1 = open('literature_dataset.csv')
f2 = open('journal_dataset.csv')

# list of all attributes in each table
attr_list_lit = f1.readline().replace('\n', '').split(sep = ',')
attr_list_jif = f2.readline().replace('\n', '').split(sep = ',')

print('Attributes of the literature table: ' + str(attr_list_lit))
print('Attributes of the journals table: ' + str(attr_list_jif))

if not import_done:
    try:
        # create table for literature dataset
        # get list of attributes
        attr_list = ''
        for i in attr_list_lit:
            attr_list += '{} varchar, '.format(i)
        attr_list = attr_list[:-2] # entfernt ', ' am Ende der Liste

        # create literature table & import data
        sql_create = 'CREATE TABLE literature ( {} );'.format(attr_list)
        print(sql_create)
        cur.execute(sql_create)
        cur.copy_expert("""COPY literature FROM STDIN WITH CSV HEADER DELIMITER AS ','""", f1)
        
        # create table for journal dataset
        # get list of attributes        
        attr_list = ''
        for i in attr_list_jif:
            attr_list += '{} varchar, '.format(i)
        attr_list = attr_list[:-2]
        
        # create impact factors table & import data
        sql_create = 'CREATE TABLE journals ( {} );'.format(attr_list)
        print(sql_create)
        cur.execute(sql_create)
        cur.copy_expert("""COPY journals FROM STDIN WITH CSV HEADER DELIMITER AS ','""", f2)
    
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)

Attributes of the literature table: ['Titel', 'Autor', 'Jahr', 'Journal', 'Typ', 'DOI']
Attributes of the journals table: ['Title', 'Rank', 'Impact_Factor', 'IF_5_Yr', 'Half_Life', 'Percentage_Citable', 'Avg_IF_Percentile']


## Visualize data

In [5]:
# show joined table
sql_showtables = """
    SELECT *
    FROM literature L, journals J
    WHERE L.Journal = J.Title;
    """

In [6]:
try:
    cur.execute(sql_showtables)
    joined = cur.fetchall()
    joined = pd.DataFrame(data = joined, columns = attr_list_lit + attr_list_jif)

except (Exception, psycopg2.DatabaseError) as error:
    print(error)

In [7]:
joined.head(5)

,Titel,Autor,Jahr,Journal,Typ,DOI,Title,Rank,Impact_Factor,IF_5_Yr,Half_Life,Percentage_Citable,Avg_IF_Percentile
0,implementation of local area vr environment us...,"kim, soo-kyun; lee, chang-hee; kim, sun-jeong;...",2020,intelligent automation and soft computing,None,10.31209/2019.100000131,intelligent automation and soft computing,8692,1.276,1.177,4.0,100.0,23.999
1,distinction between real faces and photos by a...,"lee, byong kwon; lee, yang sun",2020,intelligent automation and soft computing,None,10.31209/2019.100000134,intelligent automation and soft computing,8692,1.276,1.177,4.0,100.0,23.999
2,cognitive load and performance in immersive vi...,"frederiksen, joakim grant; sorensen, stine may...",2020,surgical endoscopy and other interventional te...,None,10.1007/s00464-019-06887-8,surgical endoscopy and other interventional te...,2996,3.149,3.239,6.5,89.56,79.286
3,tracking attacks on virtual reality systems,"rafique, muhammad usman; cheung, sen-ching s.",2020,ieee consumer electronics magazine,None,10.1109/MCE.2019.2953741,ieee consumer electronics magazine,1870,4.016,3.148,2.6,100.0,79.616
4,advantages and limits of virtual reality in le...,"checa, david; bustillo, andres",2020,virtual reality,None,10.1007/s10055-019-00389-7,virtual reality,2304,3.634,2.941,6.4,91.67,75.081


In [8]:
print('Total rows after joining: ' + str(len(joined)))

Total rows after joining: 862


In [23]:
# can use after getting an error
# conn.rollback()

## Define queries

### Sorting
#### Sort by rank

In [9]:
order = 'ASC'
# order = 'DESC'

sql_sortrank = """
    SELECT L.Titel, Autor, J.Title, CAST (Rank AS INT), Impact_Factor, IF_5_Yr
    FROM literature AS L, journals AS J
    WHERE L.Journal = J.Title
    ORDER BY Rank {};
    """.format(order)

print(sql_sortrank)


    SELECT L.Titel, Autor, J.Title, CAST (Rank AS INT), Impact_Factor, IF_5_Yr
    FROM literature AS L, journals AS J
    WHERE L.Journal = J.Title
    ORDER BY Rank ASC;
    


#### Sort by year

In [10]:
# order = 'ASC'
order = 'DESC'

sql_sortyear = """
    SELECT L.Titel, Autor, CAST (Jahr AS INT), J.Title, Rank, IF_5_Yr
    FROM literature AS L, journals AS J
    WHERE L.Journal = J.Title
    ORDER BY Jahr {};
    """.format(order)

print(sql_sortyear)


    SELECT L.Titel, Autor, CAST (Jahr AS INT), J.Title, Rank, IF_5_Yr
    FROM literature AS L, journals AS J
    WHERE L.Journal = J.Title
    ORDER BY Jahr DESC;
    


### Filtering
#### Papers published after/during/before a certain year

In [11]:
filter_op = '='
# filter_op = '<='
# filter_op = '>='
filter_year = '2018'

sql_filteryear = """
    SELECT L.Titel, Autor, Jahr, J.Title, Rank, IF_5_Yr
    FROM literature AS L, journals AS J
    WHERE L.Journal = J.Title
    AND CAST (Jahr AS INT) {} {};
    """.format(filter_op, filter_year)

print(sql_filteryear)


    SELECT L.Titel, Autor, Jahr, J.Title, Rank, IF_5_Yr
    FROM literature AS L, journals AS J
    WHERE L.Journal = J.Title
    AND CAST (Jahr AS INT) = 2018;
    


#### Papers written by a certain author (filter using last names)

In [12]:
filter_author = 'steinicke'

sql_filterauthor = """
    SELECT L.Titel, Autor, J.Title, Rank, Impact_Factor, IF_5_Yr
    FROM literature AS L, journals AS J
    WHERE L.Journal = J.Title
    AND Autor LIKE '%{}%';
    """.format(filter_author)

print(sql_filterauthor)


    SELECT L.Titel, Autor, J.Title, Rank, Impact_Factor, IF_5_Yr
    FROM literature AS L, journals AS J
    WHERE L.Journal = J.Title
    AND Autor LIKE '%steinicke%';
    


#### Papers published in a journal within a certain ranking

In [13]:
filter_rank = '500'

sql_filterrank = """
    SELECT L.Titel, Autor, Jahr, J.Title, Rank, IF_5_Yr
    FROM literature AS L, journals AS J
    WHERE L.Journal = J.Title
    AND CAST (Rank AS INT) <= {};
    """.format(filter_rank)

print(sql_filterrank)


    SELECT L.Titel, Autor, Jahr, J.Title, Rank, IF_5_Yr
    FROM literature AS L, journals AS J
    WHERE L.Journal = J.Title
    AND CAST (Rank AS INT) <= 500;
    


## Execute query

In [18]:
# enter query to execute
sql_query = sql_filterauthor

In [19]:
try:
    df = pd.read_sql_query(sql_query, conn)

except (Exception, psycopg2.DatabaseError) as error:
    print(error)

## Display results

In [20]:
# display part of the results
numrows = 10
df.head(numrows)

# display all
# df

,titel,autor,title,rank,impact_factor,if_5_yr
0,in the blink of an eye - leveraging blink-indu...,"langbehn, eike; steinicke, frank; lappe, marku...",acm transactions on graphics,1115,5.084,5.51
1,natural perspective projections for head-mount...,"steinicke, f; bruder, g; kuhl, s; willemsen, p...",ieee transactions on visualization and compute...,1444,4.558,4.317
2,change blindness phenomena for virtual reality...,"steinicke, f; bruder, g; hinrichs, k; willemse...",ieee transactions on visualization and compute...,1444,4.558,4.317
3,geometric calibration of head-mounted displays...,"kellner, f; bolte, b; bruder, g; rautenberg, u...",ieee transactions on visualization and compute...,1444,4.558,4.317
4,who turned the clock? effects of manipulated z...,"schatzschneider, christian; bruder, gerd; stei...",ieee transactions on visualization and compute...,1444,4.558,4.317
5,exploiting perceptual limitations and illusion...,"bruder, g; steinicke, f; bolte, b; wieland, p;...",displays,6800,1.738,1.645


## Close the connection

In [21]:
try:
    conn.commit()

except (Exception, psycopg2.DatabaseError) as error:
    print(error)

In [22]:
cur.close()
print('connection closed')

connection closed
